In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def generateRiblets(d, h, theta, l, d_frac):
    '''generates the vertices for a single riblet.  returns a list of 3D cartesian coords.
    d: riblet spacing
    h: riblet height
    theta: sharpness of riblet
    l: height of the system (distance to the lid driving the flow)
    d_frac: #the fraction of the separation used as the base for the trapezoid forming a side of the prism riblet'''
    w = h*math.tan(math.radians(theta/2)) #half of the base of the riblet triangle
    z = 0.1 #the length of the system in the z-direction.  Currently just used to make a dummy dimension for OpenFOAM's 2D quirks
    
    vertices = [None]*6 #list to hold all the vertices
    vertices[0] = (0, 0, 0) #origin
    vertices[1] = ((1.-d_frac)*d, 0, 0) #RHS edge of first cuboid
    vertices[2] = (d, 0, 0) #base of riblet LHS
    vertices[3] = (d+2.*w, 0, 0) #base of riblet RHS
    vertices[4] = ((1.+d_frac)*d + 2.*w, 0, 0) #LHS of edge cuboid
    vertices[5] = (2.*(d+w), 0, 0) #RHS of system base
    
    vertices = [np.array(v) for v in vertices]
    
    vertices_h = [None]*5 #the next row of vertices at the height of the riblet peak
    vertices_h[0] = (0, h, 0)
    vertices_h[1] = ((1.-d_frac)*d, h, 0)
    vertices_h[2] = (d+w, h, 0)
    vertices_h[3] = ((1.+d_frac)*d + 2.*w, h, 0)
    vertices_h[4] = (2.*(d+w), h, 0)
    
    vertices_h = [np.array(v) for v in vertices_h]
    vertices_l = [v + np.array([0, (l-h), 0]) for v in vertices_h] #the top row of vertices at the top edge of the system
    
    vertices.extend(vertices_h)
    vertices.extend(vertices_l)
    
#     for v in vertices:
#         plt.scatter(v[0], v[1])
#     plt.show()
    
    #vertices_other = vertices.copy() #duplicate of the vertices, but we add 0.1 to the z component as OpenFOAM requires 3D always
    vertices_other = [v + np.array([0., 0., z]) for v in vertices]
    
    vertices.extend(vertices_other)
    
    return vertices

In [39]:
def generateNRiblets(N, d, h, theta, l, d_frac):
    '''generates the vertices for a single riblet.  returns a list of 3D cartesian coords.
    d: riblet spacing
    h: riblet height
    theta: sharpness of riblet
    l: height of the system (distance to the lid driving the flow)'''
    w = h*math.tan(math.radians(theta/2)) #half of the base of the riblet triangle
    z = 0.1 #the length of the system in the z-direction.  Currently just used to make a dummy dimension for OpenFOAM's 2D quirks
    d_frac = 0.4 #the fraction of the separation used as the base for the trapezoid forming a side of the prism riblet
    
    initial_vertices = generateRiblets(d, h, theta, l, d_frac)
    all_vertices = initial_vertices.copy()
    system_size = initial_vertices[-1][0] #the total length of the system
    #print(system_size)
    for i in range(N-1):
        vertices_N = [v + np.array([system_size*(i+1), 0, 0]) for v in initial_vertices]
        all_vertices.extend(vertices_N)
        
    print(len(all_vertices))
    
        
    filtered_vertices = {array.tostring(): array for array in all_vertices}.values() #remove duplicate vertices
    print(len(filtered_vertices))

    for v in filtered_vertices:
        plt.scatter(v[0], v[1])
    plt.show()
    
    for i, elem in enumerate(filtered_vertices):
    #print("("+str(elem[0])+"\t"+str(elem[1])+"\t"+str(elem[2])+")")
        print("\t({0:5.3f}  {1:5.3f}  {2:5.3f})".format(elem[0], elem[1], elem[2]) + "\t //vertex "+str(i))
    

In [6]:
#vN = generateNRiblets(N=3, d=2., h=3., theta=45., l=10., d_frac=0.4)

In [50]:
def generateNRiblets_v2(N, d, h, theta, l, d_frac):
    '''generates the vertices for multiple riblets.  returns a list of 3D cartesian coords.
    this version returns all the vertices on the bottom row, followed by the middle row, then the top row.
    then repeats for the other vertices in the z-direction.
    d: riblet spacing
    h: riblet height
    theta: sharpness of riblet
    l: height of the system (distance to the lid driving the flow)'''
    w = h*math.tan(math.radians(theta/2)) #half of the base of the riblet triangle
    z = 0.1 #the length of the system in the z-direction.  Currently just used to make a dummy dimension for OpenFOAM's 2D quirks
    d_frac = 0.4 #the fraction of the separation used as the base for the trapezoid forming a side of the prism riblet
    riblet_length = 2.*(d+w) #the total length of the riblet in the x-direction
    
    n_bottom = 6 #no. of vertices in the bottom row of a single riblet
    n_middle = 5 #no. of vertices in the middle row of a single riblet
    n_top = 5 #no. of vertices in top row of a single riblet
    
    bottom_vertices = [None]*(n_bottom*N) #make the lists slightly too big then remove the Nones later on
    middle_vertices = [None]*(n_middle*N) 
    
    #fill up the vertices repeating, then remove duplicates
    for i in range(N):
        #bottom vertices
        bottom_vertices[0+i*n_bottom] = (0 + i*riblet_length, 0, 0) #origin
        bottom_vertices[1+i*n_bottom] = ((1.-d_frac)*d + i*riblet_length, 0, 0) #RHS edge of first cuboid
        bottom_vertices[2+i*n_bottom] = (d + i*riblet_length, 0, 0) #base of riblet LHS
        bottom_vertices[3+i*n_bottom] = (d+2.*w + i*riblet_length, 0, 0) #base of riblet RHS
        bottom_vertices[4+i*n_bottom] = ((1.+d_frac)*d + 2.*w + i*riblet_length, 0, 0) #LHS of edge cuboid
        bottom_vertices[5+i*n_bottom] = (2.*(d+w) + i*riblet_length, 0, 0) #RHS of system base
        
        #middle vertices
        middle_vertices[0+i*n_middle] = (0 + i*riblet_length, h, 0)
        middle_vertices[1+i*n_middle] = ((1.-d_frac)*d + i*riblet_length, h, 0)
        middle_vertices[2+i*n_middle] = (d+w + i*riblet_length, h, 0)
        middle_vertices[3+i*n_middle] = ((1.+d_frac)*d + 2.*w + i*riblet_length, h, 0)
        middle_vertices[4+i*n_middle] = (2.*(d+w) + i*riblet_length, h, 0) 
        
    #print(bottom_vertices)
    bottom_vertices = [np.array(v) for v in bottom_vertices if v]    
    middle_vertices = [np.array(v) for v in middle_vertices if v]
    
    top_vertices = [v + np.array([0, (l-h), 0]) for v in middle_vertices] #the top row of vertices at the top edge of the system
    
    all_vertices = bottom_vertices
    all_vertices.extend(middle_vertices)
    all_vertices.extend(top_vertices)
    
    vertices_other = [v + np.array([0., 0., z]) for v in all_vertices] #add the other vertices in the z-direction
    all_vertices.extend(vertices_other)
    
    print(len(all_vertices))
    filtered_vertices = list({array.tostring(): array for array in all_vertices}.values()) #remove duplicate vertices
    print(len(filtered_vertices))
    
#     plt.figure(figsize=(20,10))
#     for v in filtered_vertices:
#         plt.scatter(v[0], v[1])
#     plt.show()
    
    #print(filtered_vertices[6*N-(N-1)-1])
    for i, elem in enumerate(filtered_vertices):
    #print("("+str(elem[0])+"\t"+str(elem[1])+"\t"+str(elem[2])+")")
        print("\t({0:5.3f}  {1:5.3f}  {2:5.3f})".format(elem[0], elem[1], elem[2]) + "\t //vertex "+str(i))

In [52]:
#vN = generateNRiblets_v2(N=3, d=2., h=3., theta=45., l=10., d_frac=0.4)

In [72]:
def generateHexBlocks(N):
    '''method to generate hex blocks for N prism riblets for use in openfoam.
    N = no. of riblets'''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    hex_list = [] #this is the list that will hold all the hex blocks
    
    for i in range(N):
        bot_offset = 5*i #this moves everything along properly when we move to the next riblet
        mid_offset = 4*i
        top_offset = 4*i
        
        #the lower row of blocks, along the riblet surface
        block1 = [0+bot_offset, 1+bot_offset, nb+1+mid_offset, nb+mid_offset, 0+bot_offset+z0, 1+bot_offset+z0, nb+1+mid_offset+z0, nb+mid_offset+z0]
        block2 = [1+bot_offset, 2+bot_offset, nb+2+mid_offset, nb+1+mid_offset, 1+bot_offset+z0, 2+bot_offset+z0, nb+2+mid_offset+z0, nb+1+mid_offset+z0]
        block3 = [3+bot_offset, 4+bot_offset, nb+3+mid_offset, nb+2+mid_offset, 3+bot_offset+z0, 4+bot_offset+z0, nb+3+mid_offset+z0, nb+2+mid_offset+z0]
        block4 = [4+bot_offset, 5+bot_offset, nb+4+mid_offset, nb+3+mid_offset, 4+bot_offset+z0, 5+bot_offset+z0, nb+4+mid_offset+z0, nb+3+mid_offset+z0]
        
        
        
        hex_list.append(block1)
        print(block1)
        print(block2)
        print(block3)
        print(block4)
        print()
    #print(hex_list)
        

In [73]:
generateHexBlocks(2)

[0, 1, 12, 11, 29, 30, 41, 40]
[1, 2, 13, 12, 30, 31, 42, 41]
[3, 4, 14, 13, 32, 33, 43, 42]

[5, 6, 16, 15, 34, 35, 45, 44]
[6, 7, 17, 16, 35, 36, 46, 45]
[8, 9, 18, 17, 37, 38, 47, 46]

